In [1]:
import requests
import json
import warnings
import pandas as pd
from datetime import datetime, timedelta, date
from pprint import pprint

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# OBTENCIÓN DE DATOS DE INTERCAMBIO 

In [2]:
lang = input('¿En español (es) o en inglés (en)?')
restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [3]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [4]:
def intercambio_datos(lang, restaDia, input_año):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()



    for dato in data['included']:
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)


                fecha =  fecha.strftime("%d/%m/%Y")


                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)



    df_intercambio = pd.DataFrame()
    df_intercambio['nombre']               = lista_nombres
    df_intercambio['tipo de intercambio']      = lista_tipos
    df_intercambio['Valores']              = lista_valores
    df_intercambio["Porcentaje"]           = lista_porcentajes
    df_intercambio["Fecha actualización"]  = lista_fechas



    return df_intercambio

df_intercambio = intercambio_datos(lang, restaDia, input_año)

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-9297.7,0.166548,12/10/2024
1,Francia,Exportación,-8923.1,0.139081,13/10/2024
2,Francia,Exportación,-7027.8,0.127264,14/10/2024
3,Francia,Importación,46528.2,0.833452,12/10/2024
4,Francia,Importación,55234.3,0.860919,13/10/2024
5,Francia,Importación,48194.2,0.872736,14/10/2024
6,Francia,saldo,37230.5,1.000000,12/10/2024
7,Francia,saldo,46311.2,1.000000,13/10/2024
8,Francia,saldo,41166.4,1.000000,14/10/2024
9,Portugal,Exportación,-51614.9,0.745356,12/10/2024


In [5]:
df_intercambio.to_csv('intercambio_electrico.csv')

In [6]:
fig = px.bar(df_intercambio, x = 'nombre', y = 'Valores', color = 'tipo de intercambio',
             title= 'Valor del intercambio por País y Tipo',
             labels= {'Valores' : 'Valor de intercambio', 'nombre' : 'nombre'},
             barmode= 'group')

fig.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [7]:
fig = px.scatter(df_intercambio, x = 'Valores', y = 'Porcentaje', color = 'nombre',
                 size= 'Porcentaje', hover_name= 'tipo de intercambio',
                 title = 'Relación entre el valor del intercambio y el porcentaje de cambio',
                 size_max= 10)

fig.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [8]:
fig = px.line(df_intercambio, x = 'Fecha actualización', y = 'Valores', color= 'nombre',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por País y Tipo',
              markers= True)

fig.show()

In [9]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Francia'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio Francia',
              markers= True)

fig.show()

In [10]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Portugal'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Portugal',
              markers= True)

fig.show()

In [11]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Marruecos'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Marruecos',
              markers= True)

fig.show()

In [12]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Andorra'],
              x='Fecha actualización', y='Valores', color='tipo de intercambio',
              line_group='tipo de intercambio',
              title='Evolución de los valores de intercambio por Andorra',
              markers=True,
              line_dash='tipo de intercambio')

fig.show()
